In [ ]:
import pandas as pd
import tsfresh

In [ ]:
X = pd.read_hdf('data/achat_filtered.h5')
y = pd.read_hdf('data/features/achat_y.h5')
y.reset_index(inplace=True, drop=True)
X_filt = tsfresh.feature_selection.select_features(X,y, ml_task='classification')

In [45]:
good_sigs = [42,3,56,47,7,2,57,52]
good_features = []
for col in good_sigs:
    good_features += list(X.columns[X.columns.str.startswith(str(col))])

In [46]:
X_good = X[good_features]

In [54]:
subject = 0
for i in range(1,6):
    X.loc[(i-1)*90:i*90, 'subject'] = subject
    subject += 1

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
455    NaN
456    NaN
457    NaN
458    NaN
459    NaN
Name: subject, Length: 460, dtype: float64

In [47]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

import numpy as np
rf = RandomForestClassifier(n_estimators = 200, max_depth=3, random_state=0)
print('\n RANDOM FOREST \n Mean 3-fold cross-validation score = '+str(np.mean(cross_val_score(rf, X_good, y, cv=5))))


 RANDOM FOREST 
 Mean 3-fold cross-validation score = 0.49782608695652175


In [ ]:
import re
import os

file = 'data/features/achat_0_eff.h5'
s = os.path.split(file)[1]
electrode = re.search('achat_(.*)_eff',s)
print(electrode.group(1))

length = X.columns[X.columns.str.endswith('_change')][0]
idx_to_remove = list(X[X[length] == 1].index)
X = X.drop(idx_to_remove)
# saving potentially useful code here
# # look for rows in X where len == 1
# # this is needed due to a quirk in the data processing functions - some of the time windows end up only including 1 step (1ms)
# idx_to_remove = list(X[X[electrode+'__length'] == 1].index)
# X = X.drop(idx_to_remove)
# y = y.drop(idx_to_remove)